# Merging fitness and stability scores for epistasis modeling

In this notebook, I take 

In [1]:
import pandas as pd
import numpy as np

import dms_variants.binarymap
import dms_variants.globalepistasis

In [2]:
import pickle
def from_pickle_file(path):
    with open(path, "rb") as file:
        return pickle.load(file)

[aa_func_scores, wtseq] = from_pickle_file('../data/gb1_fitness.pkl')
[aa_stab_scores, wtseq] = from_pickle_file('../data/gb1_stability.pkl')

In [3]:
aa_func_scores.head()

,aa_substitutions,input_count,sele_count,func_score
0,Q2A,14663,38476,1.517930
1,Q2C,13001,23023,1.024400
2,Q2D,11488,18085,0.910665
3,Q2E,9501,15629,0.951582
4,Q2F,4770,13332,1.616819


In [4]:
aa_stab_scores.drop(['n_aa_substitutions'], axis=1, inplace=True)
aa_stab_scores.head()

,aa_substitutions,stability_score_sd,stability_score
0,A20D,0.16780,-1.646
1,A20E,0.04353,-0.582
2,A20F,0.09556,-1.732
3,A20G,0.17730,-1.107
4,A20H,0.14330,-1.914


drop 'Q2X' from func_scores, to make sure the wt seq lines up for both datasets

In [5]:
filter = aa_func_scores['aa_substitutions'].str.contains('Q2[A-Z]')
aa_func_scores = aa_func_scores[~filter]
aa_func_scores.head()

,aa_substitutions,input_count,sele_count,func_score
19,Y3A,24572,24579,0.578640
20,Y3C,4869,5187,0.616256
21,Y3D,11538,190,0.009526
22,Y3E,7916,121,0.008842
23,Y3F,12855,23428,1.054260


In [6]:
gb1_full = pd.merge(aa_func_scores,
                    aa_stab_scores,
                    how='outer',
                    on=['aa_substitutions']
                   )

In [7]:
gb1_full.info()
gb1_full.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517493 entries, 0 to 517492
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   aa_substitutions    517493 non-null  object 
 1   input_count         517459 non-null  float64
 2   sele_count          517459 non-null  float64
 3   func_score          517459 non-null  float64
 4   stability_score_sd  935 non-null     float64
 5   stability_score     935 non-null     float64
dtypes: float64(5), object(1)
memory usage: 27.6+ MB


,aa_substitutions,input_count,sele_count,func_score,stability_score_sd,stability_score
0,Y3A,24572.0,24579.0,0.578640,0.0000,-4.0000
1,Y3C,4869.0,5187.0,0.616256,NaN,NaN
2,Y3D,11538.0,190.0,0.009526,0.0000,-4.0000
3,Y3E,7916.0,121.0,0.008842,0.0000,-4.0000
4,Y3F,12855.0,23428.0,1.054260,0.1211,0.5499


In [8]:
# report WT seq as defined by Nisthal et al. (with T2 instead of Q2)
wtseq = "MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"

In [9]:
# export fitness data and wtseq
import pickle
with open("../data/gb1_full.pkl", "wb") as f:
      pickle.dump([gb1_full, wtseq], f)